# Resources

* University of Bremen AMSR2 snow depth retrieval:
    * [Online data viewer](https://seaice.uni-bremen.de/databrowser/#day=29&month=6&year=2016&img={"image":"image-1","sensor":"AMSRsnow","type":"nic","region":"antarctic"})
    * [Data access](https://seaice.uni-bremen.de/data/amsr2/SnowDepth/)
* Published snow depth maps: Markus and Cavalieri ([1998](http://dx.doi.org/10.1029/AR074p0019) and [2006](https://www.cambridge.org/core/product/identifier/S0260305500254670/type/journal_article))
* For an overview of the current state of snow-on-sea-ice modeling, Olivier Lecomte’s 2014 [dissertation](http://www.climate.be/php/users/lecomte/nouveau_site/images/Thesis_O_Lecomte_2014.pdf) is a comprehensive reference. He’s on the developing team for the LIM snow/sea-ice model.

# General info

* Snow-on-sea-ice loss processes are important for the ocean. As an example, a 0.3-m-thick snow layer over 2 m of sea ice halves the conductive heat flux compared to that through bare ice of the same thickness. This affects the ability of sea ice to grow and melt.
* There are substantial uncertainties in snow-on-sea-ice processes, largely due to a lack of observations.
* The three most important loss processes for snow on sea ice are probably:
    1. Aeolian (wind) transport and subsequent mass loss to leads ("lead trapping")
    2. Surface sublimation
    3. Snow-ice formation (flooding of snow on sea ice and subsequent freezing of the resultant snow-seawater slush), which is more ubiquitous around Antarctica
* Blowing snow will also result in redistribution, preferentially along microrelief and ridges in regions of deformed sea ice.

# Basic theory and parameterization

## Snowfall

Snowfall can be estimated from reanalysis total precipitation (units are often meters water equivalent per sec). When ice is present, one can consider snow height tendency as the following:
$$ \frac{dh_s}{dt} = P_{tot} * \frac{\rho_w}{\rho_s} $$  
where $\rho_w$ = 1000 kg/m$^3$ and $\rho_s$ = 330 kg/m$^3$ ($\rho_s$ from [Massom et al. 2001](http://dx.doi.org/10.1029/2000RG000085)). This is a simplification – a slight overestimate – since partitioning between liquid and solid precip occurs between about -20°C and 10°C, with approximately linear dependence on air temperature ([Ledley 1985](http://onlinelibrary.wiley.com/doi/10.1029/JD090iD01p02251/full), [Fichefet and Morales Maqueda 1999](http://link.springer.com/10.1007/s003820050280)). Note that biases in the solid/liquid partitioning may exist in atmospheric reanalysis, such as ERA-Interim ([Merkouriadi et al. 2017](http://dx.doi.org/10.1002/2017GL075494)).

## Aeolian transport

Aeolian transport depends on the balance between suspension + saltation and gravitational settling.

The following parameterization is adapted from [Leonard and Maksym 2011](https://www.cambridge.org/core/product/identifier/S0260305500252531/type/journal_article) and references therein, and incorporates key processes:

1. Initiation of dry snow transport by saltation is well-characterized as occurring over a threshold wind speed (or friction velocity). This threshold varies with ambient temperature due to changes in cohesion and frictional forces during particle collisions. [Li and Pomeroy 1997](http://journals.ametsoc.org/doi/abs/10.1175/1520-0450(1997)036%3C0205:EOTWSF%3E2.0.CO;2) give an empirical formula for this threshold: $ u_{thresh} = a + b*T + c*T^2 $, where $u_{thresh}$ is the threshold 10-m wind speed, $T$ is 2-m air temperature (°C), $a$ = 9.43 m/s, $b$ = 0.18 m °C$^{-1}$ s$^{-1}$, $c$ = 0.0033 m °C$^{-2}$ s$^{-1}$. When $u_{10m} > u_{thresh}$, one can consider aeolian transport active.
2. [Leonard and Maksym 2011](https://www.cambridge.org/core/product/identifier/S0260305500252531/type/journal_article) demonstrate that aeolian transport is likely supply-limited by the amount of unconsolidated new snow. Soon after deposition, the new snow layer becomes strongly bonded due to rapid sintering, wind crust formation, other metamorphic processes, seawater and brine wicking (capillary suction), and occasional rain penetration and re-freezing ([Thomas 2017](http://dx.doi.org/10.1002/9781118778371), ch. 3). These diagenetic processes are characterized by [Sturm et al. 1998](http://doi.wiley.com/10.1029/AR074p0001) and termed "lock-up." Leonard and Maksym 2011 account for lock-up by only allowing snow deposited in the past 2 days to be eroded, which seems justified by their data. From what I've found, snow evolution is quite sensitive to this lock-up period outside ± 1 day; this is essentially a tuning knob. One could approximately assume maxloss = MIN{SUM($h_{\mathrm{accum}}), h_s(t)$)}, where $h_{\mathrm{accum}}$ is the snow accumulated during the previous 2 days, accounting for sublimation (see below) and snow-ice formation.



## Lead trapping

The following results indicate lead trapping efficiency is very high, which leads to a straightforward parameterization of $ \Delta h_s = T_{\mathrm{eff}} * \mathrm{MIN}\{\mathrm{maxloss},h_s(t)\} $, where $T_{\mathrm{eff}}$ = 0.9 - 1.0 (90% - 100% trapping efficiency).  

1. [Déry and Tremblay 2004](http://journals.ametsoc.org/doi/abs/10.1175/1520-0485(2004)034%3C0258:MTEOWR%3E2.0.CO;2) develop a detailed boundary-layer blowing snow model, and find that the "trapping efficiency" of suspended snow depends primarily on along-fetch lead size/spacing distribution, wind speed, and friction velocity. For a range of conditions, they show that leads collect between 60-100% of blowing snow (a reasonable average being 90%).
2. [Eicken et al. 1994](https://link.springer.com/article/10.1007/s00585-994-0080-x) (note: not available online) estimate snow loss of 0.004 m/day through leads under high-wind conditions in the Weddell Sea, but only account for saltation – not suspension, which increases losses.
3. [Leonard and Maksym 2011](https://www.cambridge.org/core/product/identifier/S0260305500252531/type/journal_article) report near-total loss of snow to leads during a field experiment in the Bellingshausen Sea over a period of ECMWF-forecasted 0.032 mwe snowfall. They apply a blowing-snow model to ERA-Interim accumulated snowfall around Antarctica, which severely reduces accumulation in the interior of the Weddell.  


Other lead trapping parameterizations exist:
1. [Lecomte et al. 2013](http://dx.doi.org/10.1002/jame.20039) account for lead trapping only at the time of deposition by scaling snowfall by $[1 - (1 - c_{\mathrm{ice}})^\beta] / c_{\mathrm{ice}}$, where $\beta$ = 0.6 is an arbitrary blowing snow parameter.
2. [Lecomte et al. 2015](http://dx.doi.org/10.1016/j.ocemod.2014.12.003) take a different approach. They also use an empirical wind speed threshold, but theirs is dependent on snow density (their model tries to resolve snow compaction, so this achieves something similar to the lock-up period described above). They represent lead trapping as proportional to $(1 - c_{\mathrm{ice}})$, wind speed, and an arbitrary tuning parameter, and inversely proportional to snow density.


## Sublimation

Snow mass loss occurs due to sublimation, which is more active in the Antarctic than the Arctic due to lower relative humidity and stronger winds.

Following [Fichefet and Morales Maqueda 1999](http://link.springer.com/10.1007/s003820050280):
$$ \frac{d h_s}{dt} = \frac{F_{\mathrm{lat\_ai}}}{\rho_s * [L_c + L_f]} $$  
for $h_s > 0$, where $F_{\mathrm{lat\_ai}}$ is turbulent latent heat flux, $L_c$ = 2.5e6 J/kg (snow/ice latent heat of vaporization), $L_f$ = 0.334e6 J/kg (snow/ice latent heat of fusion). This could be an overestimate due to downstream boundary-layer condensation and ice-crystal precipitation. But more likely it’s an underestimate, since it doesn’t account for sublimation of windblown snow ([Déry and Yau 2002](http://dx.doi.org/10.1029/2001JD001251/full), [Déry and Tremblay 2004](http://journals.ametsoc.org/doi/abs/10.1175/1520-0485(2004)034%3C0258:MTEOWR%3E2.0.CO;2)).